In [1]:
import pandas as pd
df = pd.read_csv('data/aita/aita-996-processed.csv', index_col=0)
df.head()

,title,body,verdict,comments,num_comments,upvotes,upvote_ratio,url,created,edited,YTA,YWBTA,NTA,YWNBTA,ESH,NAH,INFO
1,META: This sub is moving towards a value syste...,I’ve enjoyed reading and posting on this sub f...,META,['While I find this notion accurate to a degre...,6149,80921,0.92,/r/AmItheAsshole/comments/d6xoro/meta_this_sub...,2019-09-20 11:51:40,2019-11-19 23:05:02,3,0,12,0,5,0,0
2,"UPDATE, AITA for despising my mentally handica...","I'm back like I said I would be,. My [original...",UPDATE,"[""I'm glad your other family is coming through...",1972,72782,0.96,/r/AmItheAsshole/comments/azvko1/update_aita_f...,2019-03-11 11:41:02,2019-03-12 02:49:48,0,0,18,0,0,0,3
3,AITA For suing my girlfriend after she had my ...,I'll try to keep this short. I had a [1967 Imp...,Not the A-hole,"[""NTA. My husband has several old cars. I have...",2728,70803,0.98,/r/AmItheAsshole/comments/gr8bp3/aita_for_suin...,2020-05-26 18:35:45,2020-05-27 14:04:15,0,0,124,0,0,0,1
4,UPDATE: AITA for wanting to go to the funeral ...,I want to sincerely thank everyone who comment...,Update,[],2,67573,0.91,/r/AmItheAsshole/comments/cjetsa/update_aita_f...,2019-07-29 12:58:12,NaN,0,0,0,0,0,0,0
5,AITA for pretending to get fired when customer...,I am a high schooler with a weekend job at a c...,Not the A-hole,['NTA - Customers can be fucking dicks. Usuall...,3602,63523,0.92,/r/AmItheAsshole/comments/e5k3z2/aita_for_pret...,2019-12-03 12:07:53,NaN,37,0,23,0,24,2,12


In [2]:
print(df.dtypes)
print(df.isnull().sum())
print(df['verdict'].value_counts())


title            object
body             object
verdict          object
comments         object
num_comments      int64
upvotes           int64
upvote_ratio    float64
url              object
created          object
edited           object
YTA               int64
YWBTA             int64
NTA               int64
YWNBTA            int64
ESH               int64
NAH               int64
INFO              int64
dtype: object
title             0
body              0
verdict           0
comments          0
num_comments      0
upvotes           0
upvote_ratio      0
url               0
created           0
edited          489
YTA               0
YWBTA             0
NTA               0
YWNBTA            0
ESH               0
NAH               0
INFO              0
dtype: int64
verdict
Not the A-hole             785
UPDATE                      94
Asshole                     58
Everyone Sucks              20
No A-holes here             13
META                         8
Not the A-hole POO Mode      7


In [3]:
def clean_verdict(v):
    v = v.lower()
    if 'not the a' in v:
        return 'NTA'
    elif 'asshole' in v or 'yta' in v:
        return 'YTA'
    elif 'everyone sucks' in v or 'esh' in v:
        return 'ESH'
    elif 'no a-holes' in v or 'nah' in v:
        return 'NAH'
    elif 'not enough info' in v or 'info' in v:
        return 'INFO'
    else:
        return None

df['label'] = df['verdict'].apply(clean_verdict)
print(df['label'].value_counts(dropna=False))


label
NTA     792
None    108
YTA      58
ESH      20
NAH      13
INFO      5
Name: count, dtype: int64


In [ ]:
df = df[df['label'].notnull()]
df['text'] = df['title'] + ' ' + df['body'] + ' ' + df['comments'].astype(str)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer(stop_words='english', max_features=10000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report

model = MultinomialNB()
model.fit(X_train_vec, y_train)
y_pred = model.predict(X_test_vec)

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         ESH       0.00      0.00      0.00         6
         NAH       0.00      0.00      0.00         1
         NTA       0.90      1.00      0.95       160
         YTA       0.00      0.00      0.00        11

    accuracy                           0.90       178
   macro avg       0.22      0.25      0.24       178
weighted avg       0.81      0.90      0.85       178



/opt/homebrew/Caskroom/miniconda/base/envs/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/homebrew/Caskroom/miniconda/base/envs/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av